## Generating Real Estate Listings with an LLM

In [1]:
OPENAI_API_KEY = ''

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

### Generating Real Estate Listings with an LLM

In [8]:
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, 
    model='gpt-4-turbo-preview', 
    temperature=0.5,
    max_tokens=2500)

prompt = '''
Generate 25 diverse and realistic real estate listings in the United States.
Generated listings should be catchy and descriptive, so they encourage reviewers to buy such property.
Also they should contain facts, such as:
- price
- location
- available surface
- number of bedrooms and bathrooms
- property type
- amenities
- description of the neighborhood.

Separate listings by '---'.     
'''


In [9]:
result = llm.invoke(prompt)

In [10]:
print(result.content)

1. **Charming Suburban Retreat** - $350,000 - Located in the heart of Plano, Texas, this 2,000 sq. ft. single-family home boasts 4 bedrooms and 2.5 bathrooms. Enjoy a modern kitchen, spacious backyard, and a cozy fireplace. Plano offers top-rated schools, lush parks, and a family-friendly atmosphere. 

---

2. **Modern Downtown Loft** - $475,000 - Experience urban living in this sleek loft in Seattle, Washington. With 1,100 sq. ft., 2 bedrooms, and 2 bathrooms, this property features floor-to-ceiling windows, a chef's kitchen, and communal rooftop. Steps away from vibrant nightlife, shopping, and dining.

---

3. **Historic Charleston Single** - $650,000 - Own a piece of history in Charleston, South Carolina. This 2,500 sq. ft. home offers 3 bedrooms, 3 bathrooms, original hardwood floors, and a private garden. Located in a picturesque neighborhood, walking distance to the city's best restaurants and parks.

---

4. **Beachfront Oasis** - $1.2M - Wake up to ocean views in this stunning

In [11]:
with open('listings.txt', 'w') as listings_file:
    listings_file.write(result.content)

## Semantic Search

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

### Creating a Vector Database and Storing Listings

In [13]:
loader = TextLoader("./listings.txt")
files = loader.load()

In [15]:
text_splitter = CharacterTextSplitter(separator='\n---\n', chunk_size=400, chunk_overlap=0)
docs = text_splitter.split_documents(files)
len(docs)

25

In [16]:
embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-large")
db = Chroma.from_documents(docs, embedding_model, persist_directory='./chroma_db')

/home/kcymerys/Tools/python-envs/ai/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Semantic Search of Listings Based on Buyer Preferences

In [17]:
# Test
query = '''
I would like to live in one of state in the middle of US and I can spend up to $1.5 Million. 
I want to have an estate in gated community with a lot of amenities for children (e.g. school, playground).
Beside of that house should have at least 4 bedrooms, spacious kitchen and garden.
'''
docs = db.similarity_search(query)

for doc in docs:
    print(doc.page_content)

10. **Elegant Estate** - $2.9M - Located in Greenwich, Connecticut, this luxurious 5,000 sq. ft. estate offers 6 bedrooms, 5.5 bathrooms, a gourmet kitchen, home theater, and a heated pool. Set on 2 acres of beautifully landscaped grounds in a prestigious neighborhood.
20. **Golf Course Estate** - $1.3M - Situated on a premier golf course in Palm Springs, California, this 4,500 sq. ft. estate offers 5 bedrooms, 4.5 bathrooms, a chef's kitchen, and a pool. Enjoy resort-style living with stunning views and amenities.
7. **Spacious Family Home** - $425,000 - Located in a desirable neighborhood in Orlando, Florida, this 2,800 sq. ft. home offers 5 bedrooms, 3 bathrooms, a large kitchen, and a swimming pool. Near top-rated schools and theme parks, making it an ideal family home.
11. **Rustic Ranch** - $750,000 - Enjoy country living in Bozeman, Montana, with this 3,500 sq. ft. ranch home featuring 4 bedrooms, 3 bathrooms, a large barn, and over 20 acres of land. Perfect for horse lovers and

## Augmented Response Generation

### Logic for Searching and Augmenting Listing Descriptions

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [29]:
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, 
    model='gpt-4-turbo-preview', 
    temperature=0,
    max_tokens=256)

chat_memory = ConversationBufferMemory(ai_prefix="AI Assistant")

In [30]:
template = """
You're AI asistant and your task is to gather all details from customer who is interested in buying new estate. 

At the begining, shortly describe what is purpose of this conversation.

You should gather answers for following questions:

- What are your preferences regarding style and architecture?
- What is your preferred location?
- Do you have any requirements regarding neighborhood?
- How big should your house be?
- Do you have any requirements regarding home furnishings?
- Do you expect to have some amenities?
- What is your budget?

Don't answer for question you are asking.

Be kind and don't be pushy when customer doesn't want to answer for some question, simply skip them. 

Ask one only question at once.

Once you gathered all details, thank for all replies, summarize all relevant facts that would help to recommend the best offer and put '<END-OF-CONVERSATION>'


Current conversation:
{history}
Human: {input}
AI assistant
"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=prompt,
    llm=llm,
    verbose=False,
    memory=chat_memory
)



In [31]:
current_input = 'Hello there'

end_seq = '<END-OF-CONVERSATION>'

while True:
    response = conversation.predict(input=current_input)
    print(response.replace(end_seq, ''))
    if end_seq in response:
        break
    current_input = input('Customer:')

Hello! I'm here to assist you in finding your perfect new estate. The purpose of this conversation is to gather all the necessary details about your preferences and requirements for your new home. This will help us recommend the best options tailored to your needs. Let's get started!

What are your preferences regarding style and architecture?


Customer: I love modern and classic style


That's a great combination! Modern and classic styles can complement each other beautifully. What is your preferred location?


Customer: I'd love to live at East Coast of the uS


The East Coast of the US has some beautiful locations. Do you have any requirements regarding the neighborhood?


...

I understand, a family-friendly neighborhood is important. What amenities are you looking for in your new home?


Customer: I don't want to answer for this question


That's perfectly fine! We can move on. What is your budget for the new estate?


Customer: Up to $1.5M


Thank you for sharing all these details with me. To summarize, you're looking for a home that blends modern and classic styles, located on the East Coast of the US, within a gated community that offers plenty of leisure options. The house should have at least 4 bedrooms, 2 bathrooms, and a big kitchen, and it needs to be fully furnished. It's also important for the neighborhood to be family-friendly. Your budget for this new estate is up to $1.5M.

With all this information, I'll be able to recommend the best options that meet your needs and preferences. Thank you again for your time and for providing all the necessary details. 


In [35]:
search_query = chat_memory.chat_memory.messages[-1].content.replace(end_seq, '')
search_query

"Thank you for sharing all these details with me. To summarize, you're looking for a home that blends modern and classic styles, located on the East Coast of the US, within a gated community that offers plenty of leisure options. The house should have at least 4 bedrooms, 2 bathrooms, and a big kitchen, and it needs to be fully furnished. It's also important for the neighborhood to be family-friendly. Your budget for this new estate is up to $1.5M.\n\nWith all this information, I'll be able to recommend the best options that meet your needs and preferences. Thank you again for your time and for providing all the necessary details. "

In [36]:
embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-large")

db = Chroma(embedding_function=embedding_model, persist_directory='./chroma_db')

In [37]:
recomended_offers = db.similarity_search(search_query)
recomended_offers

[Document(page_content='10. **Elegant Estate** - $2.9M - Located in Greenwich, Connecticut, this luxurious 5,000 sq. ft. estate offers 6 bedrooms, 5.5 bathrooms, a gourmet kitchen, home theater, and a heated pool. Set on 2 acres of beautifully landscaped grounds in a prestigious neighborhood.', metadata={'source': './listings.txt'}),
 Document(page_content='14. **Classic New England Home** - $550,000 - Located in Portland, Maine, this 2,400 sq. ft. home offers 4 bedrooms, 3 bathrooms, a traditional fireplace, and a large backyard. Enjoy the charm of a historic neighborhood with modern conveniences nearby.', metadata={'source': './listings.txt'}),
 Document(page_content='7. **Spacious Family Home** - $425,000 - Located in a desirable neighborhood in Orlando, Florida, this 2,800 sq. ft. home offers 5 bedrooms, 3 bathrooms, a large kitchen, and a swimming pool. Near top-rated schools and theme parks, making it an ideal family home.', metadata={'source': './listings.txt'}),
 Document(page_

### Use of LLM for Generating Personalized Descriptions

In [46]:
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, 
    model='gpt-4-turbo-preview', 
    temperature=0.75,
    max_tokens=2000)

prompt_template = PromptTemplate.from_template('''
You're and AI assistant. Your task is to select the best matching selling offer of real estate to customer needs.

Offers:
{offers}

Customer needs:
{customer_needs}

Prepare description that will encourage customer to buy selected state. Your description should emphasize customer needs

''')


In [47]:
prompt = prompt_template.format(offers='\n---\n'.join([offer.page_content for offer in recomended_offers]), customer_needs=search_query)
response = llm.invoke(prompt)

In [48]:
print(response.content)

**Classic New England Home** - $550,000 - Located in the heart of Portland, Maine, this exquisite property perfectly marries the classic charm of New England architecture with the modern conveniences you desire. Priced well within your budget at $550,000, this home offers an exceptional value and an opportunity to invest in additional customizations, making it truly your own.

This 2,400 sq. ft. home, nestled in a historic yet family-friendly neighborhood, delivers on all your requirements with 4 spacious bedrooms and 3 well-appointed bathrooms, ensuring comfort and privacy for your family and guests. The large kitchen, the heart of this home, is ideal for your culinary adventures and family gatherings, offering both the space and the modern amenities you need.

The traditional fireplace adds a touch of timeless elegance and a cozy ambiance to the living spaces, inviting you to create memorable moments with your loved ones. The expansive backyard not only provides a safe haven for outd